## Naver sentiment movie corpu

* https://github.com/e9t/nsmc


In [1]:
%%time
!rm -f ratings_train.txt ratings_test.txt
!wget -nc https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
!wget -nc https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt

--2021-07-02 19:54:19--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: `ratings_train.txt'

ratings_train.txt   100%[===================>]  13.95M  16.2MB/s    in 0.9s    

2021-07-02 19:54:21 (16.2 MB/s) - `ratings_train.txt' saved [14628807/14628807]

--2021-07-02 19:54:21--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4893335 (4.7M) [text/plain]
Savin

In [2]:
import codecs
with codecs.open("ratings_train.txt", encoding='utf-8') as f:
    data = [line.split('\t') for line in f.read().splitlines()]
    data = data[1:]   # header 제외

In [3]:
from pprint import pprint
pprint(data[0]) # pretty print : 예쁘게 줄 같은걸 맞춰서 프린트 해줌

# 데이터 순서 : 9976970번째 데이터, comment: 아 더빙.. 진짜 짜증나네요 목소리, 0,1 : 0은 안좋다, 1은 좋다.

['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0']


In [4]:
X = list(zip(*data))[1]
y = np.array(list(zip(*data))[2], dtype=int)

# X : comment
# y : 0, 1 

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

model1 = Pipeline([
    ('vect', CountVectorizer()),
    ('mb', MultinomialNB()),
])

In [6]:
%%time
model1.fit(X, y)

CPU times: user 3.02 s, sys: 99.4 ms, total: 3.12 s
Wall time: 3.12 s


Pipeline(steps=[('vect', CountVectorizer()), ('mb', MultinomialNB())])

In [7]:
import codecs
with codecs.open("ratings_test.txt", encoding='utf-8') as f:
    data_test = [line.split('\t') for line in f.read().splitlines()]
    data_test = data_test[1:]   # header 제외

In [8]:
X_test = list(zip(*data_test))[1]
y_test = np.array(list(zip(*data_test))[2], dtype=int)

print(classification_report(y_test, model1.predict(X_test)))

              precision    recall  f1-score   support

           0       0.81      0.84      0.83     24827
           1       0.84      0.81      0.82     25173

    accuracy                           0.83     50000
   macro avg       0.83      0.83      0.83     50000
weighted avg       0.83      0.83      0.83     50000



In [9]:
model1.predict(["재밌다."])

array([1])

In [10]:
model1.predict(["재밋다."])

array([1])

In [11]:
model1.predict(["별로다"])

array([0])

In [12]:
model1.predict(["최악이다"])

array([0])

In [13]:
model1.predict(["바보같다."])

array([0])

In [14]:
model1.predict(["돈이 아깝다"])

array([0])

In [15]:
model1.predict(["돈이 아깝지 않다."])

array([0])

In [16]:
model1.predict(["망작이다."])

array([0])

In [17]:
model1.predict(["명작이다."])

array([1])

In [18]:
model1.predict(["거지같다"])

array([0])

In [19]:
model1.predict(["또 보고싶다"])

array([1])

In [20]:
model1.predict(["보고싶다"])

array([1])

In [21]:
model1.predict(["존잼"])

array([1])

In [22]:
model1.predict(["존나 재밌다"])

array([1])

In [23]:
model1.predict(["역겹다"])

array([0])

In [24]:
model1.predict(["굿"])

array([0])

## 성능개선

In [25]:
# TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

model2 = Pipeline([
    ('vect', TfidfVectorizer()),
    ('mb', MultinomialNB()),
])

In [26]:
%%time
model2.fit(X, y)

CPU times: user 3.49 s, sys: 101 ms, total: 3.59 s
Wall time: 3.61 s


Pipeline(steps=[('vect', TfidfVectorizer()), ('mb', MultinomialNB())])

In [27]:
print(classification_report(y_test, model2.predict(X_test)))


# TfidfVectorizer 는 별 도움이 안되고 있다.

              precision    recall  f1-score   support

           0       0.81      0.84      0.83     24827
           1       0.84      0.81      0.83     25173

    accuracy                           0.83     50000
   macro avg       0.83      0.83      0.83     50000
weighted avg       0.83      0.83      0.83     50000



### 형태소 분석기 사용

In [28]:
from konlpy.tag import Okt # 제일 속도가 빠른 Okt;open korean text
pos_tagger = Okt()

def tokenize_pos(doc):
    return ['/'.join(t) for t in pos_tagger.pos(doc)]

In [29]:
model3 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize_pos)), # 형태도 분석기
    ('mb', MultinomialNB()),
])

In [30]:
%%time
model3.fit(X, y)

CPU times: user 8min 3s, sys: 2.43 s, total: 8min 6s
Wall time: 7min 29s


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize_pos at 0x7fccaf353d30>)),
                ('mb', MultinomialNB())])

In [31]:
print(classification_report(y_test, model3.predict(X_test)))

              precision    recall  f1-score   support

           0       0.85      0.86      0.85     24827
           1       0.86      0.85      0.85     25173

    accuracy                           0.85     50000
   macro avg       0.85      0.85      0.85     50000
weighted avg       0.85      0.85      0.85     50000



In [32]:
model4 = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize_pos, ngram_range=(1, 2))),
    ('mb', MultinomialNB()),
])

In [33]:
%%time
model4.fit(X, y)

CPU times: user 8min 15s, sys: 2 s, total: 8min 17s
Wall time: 8min 1s


Pipeline(steps=[('vect',
                 TfidfVectorizer(ngram_range=(1, 2),
                                 tokenizer=<function tokenize_pos at 0x7fccaf353d30>)),
                ('mb', MultinomialNB())])

In [34]:
print(classification_report(y_test, model4.predict(X_test)))

              precision    recall  f1-score   support

           0       0.86      0.87      0.87     24827
           1       0.87      0.86      0.87     25173

    accuracy                           0.87     50000
   macro avg       0.87      0.87      0.87     50000
weighted avg       0.87      0.87      0.87     50000



In [35]:
model4.predict(["굿"])

array([1])

In [36]:
model4.predict(["그레잇"])

array([0])